In [ ]:
try:
    import firedrake
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
    import firedrake

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from firedrake import *
import matplotlib.pyplot as plt
import numpy as np

from firedrake.petsc import PETSc

In [ ]:
# Get current path: all relative paths that you may use for input/output start from here.
#
# - Colab:  The default path is /content, and your GDrive folder is accessible (if mounted) at /content/drive/MyDrive
#
# - local:  If running on your local machine, current_path is the position WHERE YOU LAUNCHED THE NOTEBOOK KERNEL BY 'jupyter notebook'
#           and NOT the directory in which this ipynb file is saved.
#           If you want to modify your current path, go to the terminal, stop the kernel (ctrl-C + press y), then move to the desired path by
#           cd my/desired/path/starting/from/here
#           and then launch 'jupyter notebook'.
import os
current_path = os.getcwd()
print(current_path)

# my_io_path = '/content/drive/MyDrive/Colab Notebooks/CFD2324/'
my_io_path = current_path+"/"
print(my_io_path)

---
---
# Exercise 1 - Chorin-Temam projection method

\begin{equation*}
\begin{cases}
\frac{\partial  \boldsymbol{u}}{\partial t}- \Delta \boldsymbol{u} + ( \boldsymbol{u}\cdot \nabla)  \boldsymbol{u}+ \nabla  p  = \boldsymbol{0} & {\rm in} \ \Omega, t\in(0,T),\\
\nabla \cdot \boldsymbol{u} = 0 & {\rm in} \ \Omega, t\in(0,T), \\
\boldsymbol{u} = \boldsymbol{u}_\text{in} & {\rm on} \ \Gamma_{\rm{in}}, t\in(0,T),\\
\sigma(\boldsymbol{u},p)\boldsymbol{n} = h_\text{out}\boldsymbol{n} & {\rm on} \ \Gamma_{\rm{out}}, t\in(0,T), \\
\boldsymbol{u} = \boldsymbol{0} & {\rm on} \ \partial\Omega\setminus(\Gamma_\text{in}\cup\Gamma_\text{out}), t\in(0,T),\\
\boldsymbol{u}|_{t=0} = \boldsymbol{u}_\text{Stokes} & {\rm in} \ \Omega, t=0
\end{cases}
\end{equation*}

In [ ]:
# Import mesh: set the path correctly!
mesh = Mesh(my_io_path+'meshes/cylinder-ns.msh')
fig, ax = plt.subplots()
triplot(mesh, axes=ax)
ax.legend(loc='upper left')

### Variational forms for incremental Chorin-Temam steps

In [ ]:
# Forms for step 1
# u, v          : trial and test function for the predictor problem
# u_old, p_old  : solution from previous time step
def predictor_forms(u, v, Re, dt, u_old, p_old, neumann_data, neumann_tag):
    ...
    return a, L

# Forms for step 2 (incremental)
# dp, q         : trial and test function for the (incremental) pressure problem
# u_tilde       : solution of predictor problem
def pressure_forms(dp, q, dt, u_tilde):
    ...
    return a, L

# Forms for step 3: uh and grad(ph) belong to different FE spaces
# u, v          : trial and test function for the predictor problem
# u_tilde       : solution of predictor problem
# dp            : solution of (incremental) pressure problem
def projection_forms(u, v, u_tilde, dp, dt):
    ...
    return a, L

### FE spaces, data, BCs

In [ ]:
# Function spaces (separated: no MixedFunctionSpace needed)
# The bubble enrichment of the velocity space has to be done at the finite element level

...

# Data
Re = 200
f = Constant((0., 0.))
t = 0
T = 50
dt = 0.5

x = SpatialCoordinate(mesh)

# Boundary conditions - velocity
bcsU = ( ... )

# Boundary conditions - pressure
bcsP = ( ... )

### Initial condition

In [ ]:
W = MixedFunctionSpace([V, Q])  # only for Stokes problem
uu, pp = TrialFunctions(W)
vv, qq = TestFunctions(W)

# Stokes problem
a_init = ...
... # suggestion: use a suffix (e.g. _init) in all variables related to the
... # solution of the initialization problem, to avoid name clash with other
... # variables you may define afterwards
...
# the initial condition should be stored in uh_init, ph_init

# Functions on separated space, for C-T scheme
uh_old = Function(V)
ph_old = Function(Q)
uh_old.assign(uh_init)
ph_old.assign(ph_init)

### Post-processing

In [ ]:
fig, ax = plt.subplots()
col = tripcolor(ph_old, axes=ax)
plt.colorbar(col)
plt.title('Initial pressure')

fig, ax = plt.subplots()
col = quiver(uh_old, axes=ax)
plt.colorbar(col)
plt.title('Initial velocity')

# vtk output for Paraview
basename = 'lab09_'
outfileU = File(my_io_path+"output/"+basename+"velocity.pvd")
outfileP = File(my_io_path+"output/"+basename+"pressure.pvd")
uh_old.rename("Velocity")   # this names will be used in Paraview
ph_old.rename("Pressure")
outfileU.write(uh_old)
outfileP.write(ph_old)

### Definition of steps
Definition of `LinearVariationalProblem`s and `LinearVariationalSolver`s for all C-T steps.

In [ ]:
# For variational problems
u = TrialFunction(V)
v = TestFunction(V)
dp = TrialFunction(Q)   # incremental
q = TestFunction(Q)

# Actual solution
uh = Function(V)
dph = Function(Q)       # incremental

# Step 1
...

# Step 2
...

# Step 3
...

### Time-dependent solution

In [ ]:
t_vec = np.arange(0, T+0.1*dt, dt)  # T+0.1*dt to include also T: range/arange exclude the upper bound of the range
for ii in range(1, len(t_vec)):     # start from 1 to skip t=0
    t = t_vec[ii]
    print('t =', t)
    print('||u||_{H^1} =', norm(uh_old,'H1'), '  ||p||_{L^2} =', norm(ph_old,'L2')) # to check the solution is not blowing up

    ... solve steps ...

    uh_old.assign(uh)
    ph_old.vector().dat += dph.vector().dat

    # Post-processing
    outfileU.write(uh_old, time=t)
    outfileP.write(ph_old, time=t)